<a href="https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/library/tensorflow/%E5%9C%A8colab%E9%96%8B%E5%A7%8B%E5%AD%B8%E8%B5%B7tf/00/00.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# TensorFlow 2.x 學習筆記--開篇 

> 這系列筆記是由某位萌新一時興起，  
> 從 tensorflow 2.x 去學習 DL的角度所編寫而成，  
> 希望能帶給各位讀者一些小小的收穫。  
> by.萌新  
> 
> ps. 本系列會不定期的棄坑與補坑  
> pps. 想從 keras 開始的讀者請去 [此處](https://www.google.com/?hl=zh-tw) 尋找中意的教學。 

## 關於 [TensorFlow](https://www.tensorflow.org/)
  
  
1. **高維陣列運算**  
   能使用gpu對矩陣運算進行加速的數學函式庫，  
   並包含了許多矩陣的操作方法。 
2. **向後自動微分**  
   藉由將運算式解析成多個可微分算子並使用已定義好的導數規則，  
   就能在正向算出答案後反向推導出微分結果。

### TensorFlow 2.x 與 1.x 的不同之處

**面向動態計算圖的API操作方式**  
  
在 1.x 時期，TensorFlow 都是以靜態計算圖為主體，  
那時編寫 tf 會以計算的順序創建出一張計算圖，  
再透過 session 真正執行運算，使用起來不大直觀，  
直到後期推出的 eager 模式才出現改變。
  
而 2.x 則改變過往，將動態圖做為預設，  
如此就能讓使用者具有像是平常編寫程式一樣的直觀體驗，  
完成模型後，只需在函數前面加上 `@tf.function`，  
tf 2.x 就會幫你將函數轉變成靜態圖並執行最佳化。

## 小型範例
  
> 參考於 [此處](https://wellwind.idv.tw/blog/2018/04/07/tensorflow-js-basic/)  
  
本篇範例將藉由模擬乘法函數來介紹 tf 的基礎概念

### 導入 Tensorflow

In [2]:
import tensorflow as tf

print(tf.__version__)

2.2.0-rc2


### 生成資料  
  
1. [`tf.random`](https://www.tensorflow.org/api_docs/python/tf/random) 中定義了各種亂數生成方法，在此使用 `normal` 生成亂數  
2. [`tf.math`](https://www.tensorflow.org/api_docs/python/tf/math) 中定義了基本運算包含 `add`、`multiply`、`abs` 等等，  
   也有需多基本運算可直接從 tf 底下調用，或是有實作運算子重載。  

In [0]:
def generate_data(data_num, answer):
    x = tf.random.normal((data_num,1))

    # 從 tf.math 下調用 multiply
    y = tf.math.multiply(x, answer)

    # 從 tf 下直接調用 multiply
    # y = tf.multiply(x, answer)

    # 使用重載過的運算子
    # y = x * answer

    return [x,y]

### 訂定函數
  
1. [`@tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) 代表要讓 tf 將此函數轉換成計算圖並進行最佳化  
2. [`tf.linalg`](https://www.tensorflow.org/api_docs/python/tf/linalg) 為線性代數的函示庫，內含有 `matmul`、`qr` 等等，  
   其中有部分函數可從 tf 下直接調用，或是有實作運算子重載。

In [0]:
@tf.function
def model(w,x):
    # 從 tf.linalg 下調用 matmul
    return tf.linalg.matmul(x, w)
    
    # 從 tf 下直接調用 matmul
    # return tf.matmul(x, w)
    
    # 使用重載過的運算子
    # return x @ w

### 訓練流程
  
1. [`tf.optimizers`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) 中放有各種優化器，在此使用 `Adamax` 作為優化器。  
   在計算出梯度後使用 `apply_gradients` 改變權重  
2. [`tf.losses`](https://www.tensorflow.org/api_docs/python/tf/keras/losses) 中放有各式各樣的損失函數，在此使用 `mean_squared_error` 計算 loss。  
> **$MSE = \frac{1}{n}\sum_{i=1}^n(y_i-y_i)^2$**  
3. 在 [`with tf.GradientTape() as tape:`](https://www.tensorflow.org/api_docs/python/tf/GradientTape) 下的運算將會被記錄下來，  
   之後再使用 `tape.gradient(loss, weights)` 計算出權重的梯度。

In [0]:
def train(model, w, x, y, epoch, lr=0.001):
    optimizer = tf.optimizers.Adamax(lr)
    for i in range(epoch):
        with tf.GradientTape() as tape:
            y_pred = model(w, x)
            loss = tf.losses.mean_squared_error(y_pred=y_pred, y_true=y)
        grads = tape.gradient(loss, [w])
        optimizer.apply_gradients(zip(grads, [w]))


### 開始訓練
- [`tf.Variable`](https://www.tensorflow.org/api_docs/python/tf/Variable) 製作 tensor 變數，  
  比較重要的參數有：  
  1. initial_value : 初始值
  2. trainable(預設為 True) : 是否可被 `optimizer` 更新數值
  3. shape(預設為 initial_value 的維度大小) : tensor 的維度大小
  4. dtype(預設為 initial_value 的型別) : tensor 的型別，詳細可見 [`tf.dtypes`](https://www.tensorflow.org/api_docs/python/tf/dtypes)

In [11]:
w = tf.Variable(tf.random.normal([1,1]))
tf.dtypes.
[x, y] = generate_data(10000, 20)

print("w : ", w.value().numpy())

for i in range(5):
    train(model, w, x, y, 100, 0.1)
    print("loss : ", tf.math.reduce_mean(tf.losses.mean_squared_error(y_pred=model(w,x), y_true=y)).numpy())

print("w : ", w.value().numpy())

w :  [[0.22930485]]
a


ValueError: ignored